In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [2]:
## 数据读取
train_data = pd.read_csv("./data/train.csv", index_col=0)
test_data = pd.read_csv("./data/test.csv", index_col=0)
#submission = pd.read_csv("./data/submission.csv")

train_data['subscribe'] = train_data['subscribe'].map({'yes':1, 'no':0})

In [7]:
# 区分特征大类, 进行基础处理

category_feature = [x for x in train_data.columns if train_data[x].dtype == 'object' and x != 'subscribe']
numeric_feature = [x for x in train_data.columns if train_data[x].dtype != 'object'  and x != 'id']
category_feature, numeric_feature

# 对标签类的特征, 进行变换
for col in category_feature:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col])
    test_data[col] = le.fit_transform(test_data[col])

# 对于数值类特征, 去除异常值
for x in numeric_feature:
    if x == 'subscribe': 
        continue
    temp1 = train_data[x]
    temp2 = test_data[x]
    q1 = temp1.quantile(0.25)
    q2 = temp1.quantile(0.75)
    delta = (q2 - q1) * 1.5
    train_data[x] = np.clip(temp1, q1 - delta, q2 + delta)
    test_data[x] = np.clip(temp2, q1 - delta, q2 + delta)

X_train = train_data.drop(['subscribe'], axis=1)
Y_train = train_data['subscribe']




id
1        0
2        1
3        0
4        1
5        0
        ..
22496    1
22497    0
22498    0
22499    0
22500    1
Name: subscribe, Length: 22500, dtype: int64

In [27]:
# 步骤3：用处理好的 NumPy 数组，创建一个全新的、干净的 DataFrame
processed_columns = {}
for col in X_train:
    processed_columns[col] = X_train[col].to_numpy(dtype=np.float64)
temp = pd.DataFrame(processed_columns)
X_train_final = pd.DataFrame(temp)

#
processed_columns = {}
for col in test_data:
    processed_columns[col] = test_data[col].to_numpy(dtype=np.float64)
temp = pd.DataFrame(processed_columns)
X_test_final = pd.DataFrame(temp)

test_data_with_id = test_data.reset_index()
X_test_final = pd.concat([test_data_with_id['id'], X_test_final], axis=1)

In [28]:
X_test_final

,id,age,job,marital,education,default,housing,loan,contact,month,...,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_index,cons_conf_index,lending_rate3m,nr_employed
0,22501,35.0,9.0,2.0,5.0,0.0,2.0,0.0,0.0,1.0,...,3295.0,1.0,476.0,0.0,1.0,1.4,95.37,-33.04,3.63,5204.54
1,22502,26.0,0.0,2.0,3.0,0.0,2.0,0.0,0.0,6.0,...,2872.0,1.0,166.0,2.0,1.0,-1.8,91.75,-44.42,3.16,4924.78
2,22503,44.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,6.0,...,268.0,3.0,968.0,0.0,1.0,1.1,89.67,-36.90,5.04,4947.02
3,22504,36.0,1.0,1.0,2.0,0.0,2.0,0.0,1.0,7.0,...,30.0,1.0,432.0,5.0,1.0,-0.1,89.87,-41.66,3.27,5203.33
4,22505,41.0,1.0,1.0,0.0,0.0,2.0,0.0,1.0,6.0,...,1670.0,1.0,944.0,3.0,1.0,1.1,97.64,-36.32,3.95,4992.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,29996,49.0,0.0,3.0,6.0,0.0,2.0,0.0,1.0,0.0,...,3937.0,6.0,302.0,1.0,1.0,-1.8,95.77,-40.50,3.86,5058.64
7496,29997,34.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,...,584.0,6.0,440.0,3.0,1.0,1.4,90.59,-47.29,1.77,5156.70
7497,29998,50.0,5.0,2.0,0.0,0.0,2.0,0.0,0.0,4.0,...,102.0,3.0,997.0,0.0,1.0,-2.9,97.42,-39.69,1.29,5116.80
7498,29999,31.0,9.0,1.0,5.0,0.0,0.0,0.0,0.0,1.0,...,179.0,3.0,1028.0,0.0,1.0,1.4,96.90,-37.68,5.18,5144.45


In [11]:
# 使用网格调参
learning_rate = [0.05, 0.1, 0.15, 0.2]
subsample = [0.7, 0.8, 0.9]
colsample_bytree = [0.7, 0.8, 0.9]
max_depth = [2, 3, 4, 5, 6, 7]

# --- 使用网格调参 ---
parameters = {'learning_rate': learning_rate,
              'subsample': subsample,
              'colsample_bytree': colsample_bytree,
              'max_depth': max_depth}

model = XGBClassifier(n_estimators=50, base_score=0.5)
Y_train = Y_train.astype(int)
# --- 重要调整：处理数据中的NaN值 ---
# 检查并处理X_train_final中的NaN
if isinstance(X_train_final, pd.DataFrame):
    # 将DataFrame中的NaN替换为0（或用其他策略，如均值/中位数）
    X_train_final = X_train_final.fillna(0)
elif isinstance(X_train_final, np.ndarray):
    # 将Numpy数组中的NaN替换为0
    X_train_final = np.nan_to_num(X_train_final)

# 确保Y_train中没有NaN（虽然int64类型通常不会有）
if isinstance(Y_train, pd.Series):
    Y_train = Y_train.dropna()

#print(type(Y_train), Y_train.dtypes, Y_train.shape)
#print(type(X_train_final), X_train_final.dtypes, X_train_final.shape)
#print(type(X_test_final),  X_test_final.dtypes, X_test_final.shape)

# 做出最终预测
clf = GridSearchCV(model, parameters, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
clf.fit(X_train_final, Y_train)





Fitting 3 folds for each of 216 candidates, totalling 648 fits
[0 0 0 ... 0 0 0]


In [31]:
# 产生最终结果并保存结果

result = clf.predict(X_test_final)
out = pd.DataFrame(result)
print(out.value_counts())

X_test_final['subscribe'] = result
X_test_final['subscribe'] = X_test_final['subscribe'].map({1:'yes', 0:'no'})
X_test_final



0
0    7013
1     487
Name: count, dtype: int64


,id,age,job,marital,education,default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_index,cons_conf_index,lending_rate3m,nr_employed,subscribe
0,22501,35.0,9.0,2.0,5.0,0.0,2.0,0.0,0.0,1.0,...,1.0,476.0,0.0,1.0,1.4,95.37,-33.04,3.63,5204.54,no
1,22502,26.0,0.0,2.0,3.0,0.0,2.0,0.0,0.0,6.0,...,1.0,166.0,2.0,1.0,-1.8,91.75,-44.42,3.16,4924.78,no
2,22503,44.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,6.0,...,3.0,968.0,0.0,1.0,1.1,89.67,-36.90,5.04,4947.02,no
3,22504,36.0,1.0,1.0,2.0,0.0,2.0,0.0,1.0,7.0,...,1.0,432.0,5.0,1.0,-0.1,89.87,-41.66,3.27,5203.33,no
4,22505,41.0,1.0,1.0,0.0,0.0,2.0,0.0,1.0,6.0,...,1.0,944.0,3.0,1.0,1.1,97.64,-36.32,3.95,4992.02,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,29996,49.0,0.0,3.0,6.0,0.0,2.0,0.0,1.0,0.0,...,6.0,302.0,1.0,1.0,-1.8,95.77,-40.50,3.86,5058.64,no
7496,29997,34.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,...,6.0,440.0,3.0,1.0,1.4,90.59,-47.29,1.77,5156.70,no
7497,29998,50.0,5.0,2.0,0.0,0.0,2.0,0.0,0.0,4.0,...,3.0,997.0,0.0,1.0,-2.9,97.42,-39.69,1.29,5116.80,no
7498,29999,31.0,9.0,1.0,5.0,0.0,0.0,0.0,0.0,1.0,...,3.0,1028.0,0.0,1.0,1.4,96.90,-37.68,5.18,5144.45,no


In [34]:
selected = X_test_final[['id', 'subscribe']]
selected.to_csv("./data/submission.csv", index=False)